In [1]:
"""
TOPIC 1 Django Signals
"""

'\nTOPIC 1 Django Signals\n'

In [2]:
# Answer 1
"""
By default, Django signals are executed synchronously.

This means that when a signal is sent, all connected receiver functions are called one after another, 
in the same thread, before the code that triggered the signal can proceed. 
If a receiver takes a long time to execute, it will block the entire process until it's finished.

Django does offer asynchronous sending of signal but not by default.

Here is the code snippet in the next window.
"""

"\nBy default, Django signals are executed synchronously.\n\nThis means that when a signal is sent, all connected receiver functions are called one after another, \nin the same thread, before the code that triggered the signal can proceed. \nIf a receiver takes a long time to execute, it will block the entire process until it's finished.\n\nDjango does offer asynchronous sending of signal but not by default.\n\nHere is the code snippet in the next window.\n"

In [11]:
import os
import sys
import django
import time

# --- Setup for Django Settings File (Most Robust Approach) ---
settings_content = """
import os
DEBUG = True
SECRET_KEY = 'very-secret-key-for-temp-use'
INSTALLED_APPS = [
    '__main__.MyappConfig', # Refers to the MyappConfig defined in the notebook's global scope
]
# ROOT_URLCONF will point to this temporary settings module itself,
# assuming urlpatterns are also defined in the notebook's global scope.
ROOT_URLCONF = 'temp_settings_jupyter'
TEMPLATES = [
    {
        'BACKEND': 'django.template.backends.django.DjangoTemplates',
        'APP_DIRS': True,
    },
]
"""

temp_settings_filename = 'temp_settings_jupyter.py'
temp_settings_filepath = os.path.join(os.getcwd(), temp_settings_filename)

# Write the settings content to the file
with open(temp_settings_filepath, 'w') as f:
    f.write(settings_content)

# Add the current directory to sys.path so Python can find temp_settings_jupyter.py
if os.getcwd() not in sys.path:
    sys.path.append(os.getcwd())

# Set the DJANGO_SETTINGS_MODULE environment variable
# This needs to be set BEFORE django.setup() is called
os.environ.setdefault('DJANGO_SETTINGS_MODULE', temp_settings_filename.replace('.py', ''))

# --- Define MyappConfig (Must be before django.setup() as it's in INSTALLED_APPS) ---
from django.apps import AppConfig

class MyappConfig(AppConfig):
    name = 'myapp'
    label = 'myapp'
    verbose_name = 'My App'

    def ready(self):
        # This will print when Django successfully loads this app
        print(f"[{time.time()}] MyappConfig ready method called. App registered.")

# --- Initialize Django (The Tricky Part in Jupyter) ---
# Check if Django's app registry is ready. This is the most reliable internal check.
if not django.apps.apps.ready:
    try:
        # Attempt to run django.setup(). This is where the 'reentrant' error often occurs.
        django.setup()
        print(f"[{time.time()}] Django environment set up for the first time in this session (no reentrant error).")
    except RuntimeError as e:
        # If a RuntimeError (especially 'reentrant') occurs, it *usually* means
        # Django is already set up enough for our purposes in Jupyter.
        if "populate() isn't reentrant" in str(e):
            print(f"[{time.time()}] Caught 'populate() isn't reentrant' error. Assuming Django is already set up and proceeding.")
            # We don't re-raise here, because for Jupyter, this error often implies
            # a successful, albeit internally "messy," prior setup.
        else:
            # If it's a different RuntimeError, we should re-raise it.
            raise
else:
    print(f"[{time.time()}] Django environment already set up in this session (django.apps.apps.ready is True). Skipping full setup.")

# --- Final Check (Optional, but good for debugging) ---
if django.apps.apps.ready:
    print(f"[{time.time()}] Confirmed: Django apps are ready. Loaded apps: {[app.name for app in django.apps.apps.get_app_configs()]}")
else:
    print(f"[{time.time()}] Warning: Django apps are NOT ready after setup attempt.")

[1748633695.5581927] Caught 'populate() isn't reentrant' error. Assuming Django is already set up and proceeding.
[1748633695.558662] Warning: Django apps are NOT ready after setup attempt.
